# Import Hacks

That said, we still have to manually decorate each and every function in
our code to make sure they work right.

Naturally, as we keep writing and using our C++.py code,
we'll have it in a separate package that we can import.

So no matter what we do, we'll have to import our code.
`from cpp import whatever`.

Wouldn't it be nice if we could just write `from cpp import magic`
at the first line of our code, and have all the relevant behaviour
automatically brought in?

In [ ]:
from cpp import cpp_function


@cpp_function
def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)


main()


In [ ]:
from cpp import magic

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)


Well, let's start!

First, we'll call a function at the end of our module to decorate everything:

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

magic()

main()

Our `magic()` function needs to do 2 things.
Get the module it was called in, and decorate all the functions.

In [ ]:
def magic():
    calling_module = get_calling_module()
    decorate_module_functions(calling_module)

To get the calling module, we use `inspect.stack` to traverse
the callstack and find the right module

In [ ]:
import inspect

def get_calling_module():
    # Use 2 here because we need to go 2 frames up.
    stack_frame = inspect.stack()[2].frame
    module = inspect.getmodule(stack_frame)
    return module

We get the callstack, take the frame 2 level above us (the caller to `magic()`),
and use `inspect.getmodule` to get the relevant module.

Now we need to decorate our functions!

In [ ]:
def decorate_module_functions(module):
    for name, value in inspect.getmembers(module):
        if not inspect.isroutine(value):
            continue

        # Only convert functions that were defined in the importing file.
        # We don't want to convert library imports and the likes of those.
        if inspect.getmodule(value) != module:
            continue

        setattr(module, name, cpp_function(value))

Our function takes a module and modifies it.

To do this, we're using some of Python's reflection capabilities.
`inspect.getmembers(obj)` returns all the member variables of a given object.
In our case - a module.
`inspect.isroutine(obj)` tells us whether a value is a function.
`inspect.getmodule(obj)` returns the module an object was defined in.
`setattr(obj, name, value)` sets an object attribute named `name` to `value`.

And with that our `magic()` function is operational!

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

magic()

main()

**Maybe this should be done in 2 parts?
First we handle libraries.
They are simpler to handle as we can execute them safely.
Then we handle the main module, which cannot execute more than
once and cannot be replaced.**

Great!

Now, the next step is to make the call to `magic()` disappear as well.

This means that we need to somehow make the `from cpp import magic` line
do the actual magic.

"But Tamir! An import is not a function call!" you might say.

Well, let's see what a Python import actually does.

Let's look at an example to see how imports work in Python.

When we run `from cpp import magic` we go through the following steps.

First, we look for a module named `cpp` in the global module cache,
`sys.modules`.
If it is present, the module is already loaded, and we can skip to name binding.

If it is not present in the cache, we find it on disk, place module object
in the cache, and then execute the module.

Note that we first store it in the cache, and only then execute the module.
This is important as Python does allow for cyclic imports, and we want to
avoid recursion.
It will also come in handy later.

Once we finish executing the module, we need to bind the relevant names.
In this case - `magic`.

Python takes the `cpp` module from `sys.modules` and looks for `magic`
inside it.
If it finds it, it binds that to the name `magic`.

Lastly, Python modules may define a `__getattr__(name)` function.
If it is defined, it is called whenever we try to import a name
that isn't present in the module.

In [ ]:
magic = cpp.magic

magic = cpp.__getattr__("magic")

sys.modules

So, as you can see - `import` _can_ be a function call!



In [ ]:
def _magic():
    calling_module = get_calling_module()
    decorate_module_functions(calling_module)

def __getattr__(name):
    if name != "magic":
        raise AttributeError()

    _magic()

This converts the import to a call and allows C++ to work it's magic.

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

main()

Well, almost...

You see, since we're imported on the first line of the module,
the module is empty.
The functions we want to decorate are not yet defined.

To fix this, we can do one of two things.

The first option is to import our magic at the end of the file

In [ ]:
from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

from cpp import magic

main()

This works, but feels far from magical.

The other option, then, is to import the modules ourselves!
With the fully imported module at hand, we can modify it as we wish.

This means that our `_magic()` function is going to change a little:

In [ ]:

import importlib.util
import sys

def import_by_path(name: str, path: str):
    spec = importlib.util.spec_from_file_location(name, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    return module


def _magic():
    calling_module = get_calling_module()
    name = calling_module.__name__
    path = calling_module.__file__
    imported_module = import_by_path(name, path)
    decorate_module_functions(imported_module)

Using Python's import mechanisms, we import another instance of the
module that imported us.

We use it's name and path to import it again, then store it in the
global module cache instead of the original.

This is where the fact that the cache is filled _prior_ to module execution
comes in handy!

We execute the module, to define all the types,
and then decorate the functions.

With this change, we can now write

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

main()

And recurse infinitely.

Our module runs the `magic()` function.
The `magic()` function imports our module.
The module runs the `magic()` function.
The `magic()` function imports our module.

And so on and so forth.

To fix that, we add a flag to all the modules we import,
before executing them.

Then, in our `magic()` function, we check for the flag.

In [ ]:
IMPORT_FLAG = "__magically_imported__"

def import_by_path(name: str, path: str):
    spec = importlib.util.spec_from_file_location(name, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    setattr(module, IMPORT_FLAG, True)
    spec.loader.exec_module(module)
    return module


def _magic():
    calling_module = get_calling_module()
    name = calling_module.__name__
    path = calling_module.__file__
    if hasattr(calling_module, IMPORT_FLAG):
        return

    imported_module = import_by_path(name, path)

    decorate_module_functions(imported_module)

This breaks the recursion.

Now we're good to go, except for one thing.

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

main()

Once we finish all of our import magic,
we return to the module that triggered the magic.
This module has not yet been modified.
Once we return to it, it'll run to completion.
This means that any side-effects will take place.
In the case of our main module - `main()` will run twice.
First, when we import it inside `magic()`.
Second, when we return to the main module and let it execute.
In both cases, we'll be running the non-decorated version.

To avoid this kind of thing, Python code usually uses the following:

In [ ]:
if __name__ == "__main__":
    main()

`__name__` always holds the name of the current module.
In the case of the main module, it'll be `"__main__"`.

This ensures that when a module is imported it will not
run the `main()` function.

In our case, this will not be enough.
First, we actually do import the module.
Second, we need to ensure that once we're done, the original doesn't run.

So once again, we modify our magic function!

In [ ]:
def _magic():
    calling_module = get_calling_module()
    name = calling_module.__name__
    path = calling_module.__file__
    if hasattr(calling_module, IMPORT_FLAG):
        return

    imported_module = import_by_path(name, path)

    decorate_module_functions(imported_module)

    if imported_module.__name__ == "__main__":
        sys.exit(imported_module.main())

Now, once we finished handling and decorating the main module,
we call it's `main()` function and terminate.

And with that, we can truly write our module:

In [ ]:
from cpp import magic

from greeter import Greeter

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)
